# Data Cleaning of kc_house_data.csv Data

### Import required packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.impute import SimpleImputer

For project examples, reference ISIL Machine Learning Project example as reference

### Import kc_house_data.csv to dataframe

In [2]:
!pwd

/Users/algrhythm/Flatiron/mod1/section01/Code/dc-ds-021720/mod-2/week-3/Mod2_Project/mod2_project/step1_data_cleaning


In [3]:
data = pd.read_csv('../../mod2_project/data/kc_house_data.csv')

### Review the housing data head

In [4]:
data[:3];

### Review data.info()

In [5]:
# data.info();

### Initial issues with the data inlcude:
- data.yr_renovated inlcudes 17,755 values across 21,597 rows represnting 17.8% of the data missing or 3,842 rows
- data.waterfront includes 19,221 values across 21,597 rows representing 11% of the data missing or 2,376
- data.view includes 21,534 values across 21,597 rows represnting 0.003% of the data missing or 63 rows

These nulls will updated using imputation later in this notebook.

### Describe the data dataframe and look for potential errors

In [6]:
data.describe();

There is a house with 33 bedrooms, this must be reviewed for validity.
There is a house with 8 bathrooms, this must be reviewed for validity.
There is a house with 3.5 floors, is this an attic?  What does the 0.5 signify?

There are some similar columns that are assumed to be highly corelated.  These inlcude:
- The sqft_living and sqft_lot will be very similar in that the living footage must be less than or equal to the lot itself
- The sqft_living 15 and sqft_lot15 would introduce a similar correlation

In [7]:
# problems = data.loc[(data.bedrooms == 33)]
data.loc[(data.bedrooms == 33)]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
15856,2402100895,6/25/2014,640000.0,33,1.75,1620,6000,1.0,0.0,0.0,...,7,1040,580.0,1947,0.0,98103,47.6878,-122.331,1330,4700


### Investigate the relationship between sqft lving and the lot as well as the floors
Prior precieved inequity resulted from addition of basement to living area

### Investigate data.floors data

In [8]:
data.loc[(data.sqft_living >= data.sqft_lot)].loc[(data.floors < 2)]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
1547,8816400885,10/8/2014,450000.0,4,1.75,1640,1480,1.0,0.0,0.0,...,7,820,820.0,1912,0.0,98105,47.6684,-122.314,1420,2342
5224,9828702890,2/11/2015,760000.0,5,1.50,3050,2992,1.5,0.0,0.0,...,8,1920,1130.0,1931,0.0,98112,47.6210,-122.302,1200,1209
13240,2877104196,12/6/2014,760000.0,3,2.00,1780,1750,1.0,0.0,2.0,...,8,1400,380.0,1927,2014.0,98103,47.6797,-122.357,1780,3750
13265,3277800845,7/11/2014,370000.0,3,1.00,1170,1105,1.0,NaN,0.0,...,7,1170,0.0,1965,0.0,98126,47.5448,-122.375,1380,1399
13607,7011201475,5/27/2014,780000.0,3,3.00,2520,2152,1.5,0.0,0.0,...,8,1560,960.0,1925,2006.0,98119,47.6363,-122.371,1140,2152
13861,3705000120,7/29/2014,284000.0,3,2.25,2080,2050,1.5,0.0,0.0,...,7,1550,530.0,2003,0.0,98042,47.4199,-122.157,2080,2275
15729,9828702895,10/22/2014,700000.0,4,1.75,2420,520,1.5,0.0,0.0,...,7,2420,0.0,1900,0.0,98112,47.6209,-122.302,1200,1170
15896,4310701330,3/9/2015,415000.0,3,1.50,1220,835,1.5,0.0,0.0,...,6,1220,0.0,1950,0.0,98103,47.6981,-122.341,1360,1251
16917,5016002275,6/2/2014,610000.0,5,2.50,3990,3839,1.0,NaN,0.0,...,8,1990,2000.0,1962,0.0,98112,47.6236,-122.299,2090,5000
17910,8032700175,10/27/2014,420000.0,4,1.00,1510,1501,1.5,0.0,0.0,...,7,1510,0.0,1906,0.0,98103,47.6526,-122.342,1560,1602


Floors are listed to include __0.5__. Open source research indicates this signifies a __loft or attic__.

### Create attic column where 0 is no attic and 1 is an attic

In [9]:
data['attic'] = data['floors'].apply(lambda f: 0 if f - int(f) == 0 else 1)

### Some of the values for sqft_basement are '?'

In [10]:
data.sqft_basement[data.sqft_basement == '?'].count()

454

In total, ther are 454 rows with '?' instead of numbers in the sqft_basement column.  454 rows represent 2.1% of all rows

### Data Cleaning decisions
The following decisions were made regarding the King County house price data analysis:
- the waterfront and yr_renovated columns are missing between 11% and 17.8% of their total row values and replaced with imputation
- For the view and sqft_basement, they represent far fewer missing rows and therefore these rows will be removed
- The house with 33 bedrooms will be removed

In [11]:
# Create funtion to impute by mean
def impute_mean(col):
    return col.fillna(col.mean())

In [12]:
# Impute waterfront
data.waterfront = impute_mean(data.waterfront)

In [19]:
# Impute yr_renovated
data.yr_renovated = impute_mean(data.yr_renovated)

In [20]:
# Remove null rows for view and '?' in sqft_basement
data = data.dropna()
data['sqft_basement'] = data['sqft_basement'].apply(lambda b: 0 if b == '?' else b)

In [21]:
# Drop the house (row) with 33 bedrooms
data = data.drop(data[data.bedrooms == 33].index, axis=0)

### Confirm nulls are gone

In [31]:
data.info();

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21533 entries, 0 to 21596
Data columns (total 22 columns):
id               21533 non-null int64
date             21533 non-null object
price            21533 non-null float64
bedrooms         21533 non-null int64
bathrooms        21533 non-null float64
sqft_living      21533 non-null int64
sqft_lot         21533 non-null int64
floors           21533 non-null float64
waterfront       21533 non-null float64
view             21533 non-null float64
condition        21533 non-null int64
grade            21533 non-null int64
sqft_above       21533 non-null int64
sqft_basement    21533 non-null object
yr_built         21533 non-null int64
yr_renovated     21533 non-null float64
zipcode          21533 non-null int64
lat              21533 non-null float64
long             21533 non-null float64
sqft_living15    21533 non-null int64
sqft_lot15       21533 non-null int64
attic            21533 non-null int64
dtypes: float64(8), int64(12), object

### Export data to data_cleaned.csv

In [32]:
data.to_csv('../../mod2_project/data/data_cleaned.csv')